2011년,2012년 자전거 대여 데이터  
대여 데이터는 1시간 간격으로 기록  
훈련 데이터는 19일까지, 테스트 데이터는 20일부터 월말까지  
피처:날짜,시간,요일,계절,날씨,실제 온도,체감 온도,습도.풍속,회뤈 여부  
시간별 자전거 대여 수량을 예측하면 된다  
예측할 값이 범주형이 아니므로 회귀 문제에 속함

**피처와 타깃값이란?**  
피처는 원하는 값을 예측하기 위해 활용하는 데이터를 의미  
타깃값은 예측해야 할 값  

In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submission=pd.read_csv("sampleSubmission.csv")

In [3]:
train.shape

(10886, 12)

In [4]:
test.shape

(6493, 9)

|피처명|설명|
|---|---|
|datetime|기록 일시(1시간 간격)|
|season|계절(1:봄,2:여름,3:가을,4:겨울|
|holiday|공휴일 여부(0:아님,1:공휴일)|
|workingday|근무일 여부(0:근무일 아님,1:근무일)|
|weather|날씨(1:맑음,2:옅은 안개,3:약간의 눈과 비,흐림,4:폭우와 천둥 번개,눈과 짙은 안개|
|temp|실제 온도|
|atemp|체감 온도|
|humidity|상대 습도|
|windspeed|풍속|
|casual|비회원 수|
|registered|회원 수|
|count|자전거 대여 수량|

In [5]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [6]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


casual이랑 registered이 빠짐	

In [7]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


시간대별 대여 수량을 예측해 이 값을 바꿔서 제출하면 됨 

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    6493 non-null   object 
 1   season      6493 non-null   int64  
 2   holiday     6493 non-null   int64  
 3   workingday  6493 non-null   int64  
 4   weather     6493 non-null   int64  
 5   temp        6493 non-null   float64
 6   atemp       6493 non-null   float64
 7   humidity    6493 non-null   int64  
 8   windspeed   6493 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 456.7+ KB


## 피처 엔지니어링

datetime를 연도,월,일,시간,분,초로 나누기

In [10]:
train['date']=train['datetime'].apply(lambda x: x.split()[0])
train['year']=train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month']=train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day']=train['datetime'].apply(lambda x: x.split()[0].split('-')[2])
train['hour']=train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute']=train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second']=train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

In [12]:
from datetime import datetime
import calendar
train['weekday']=train['date'].apply(lambda dateString:
                                    calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])

In [14]:
train['season']=train['season'].map({1:'spring',
                                   2:'summer',
                                   3:'fall',
                                   4:'winter'})

In [15]:
train['weather']=train['weather'].map({1:'clear',
                                       2:'mist,few clouds',
                                      3:'light snow,rain,thunderstorm',
                                      4: 'heavy rain, thunderstorm,snow,fog'})

In [17]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,year,month,day,hour,minute,second,weekday
0,2011-01-01 00:00:00,spring,0,0,clear,9.84,14.395,81,0.0,3,13,16,2011-01-01,2011,01,01,00,00,00,Saturday
1,2011-01-01 01:00:00,spring,0,0,clear,9.02,13.635,80,0.0,8,32,40,2011-01-01,2011,01,01,01,00,00,Saturday
2,2011-01-01 02:00:00,spring,0,0,clear,9.02,13.635,80,0.0,5,27,32,2011-01-01,2011,01,01,02,00,00,Saturday
3,2011-01-01 03:00:00,spring,0,0,clear,9.84,14.395,75,0.0,3,10,13,2011-01-01,2011,01,01,03,00,00,Saturday
4,2011-01-01 04:00:00,spring,0,0,clear,9.84,14.395,75,0.0,0,1,1,2011-01-01,2011,01,01,04,00,00,Saturday


## 시각화